# 层和块

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [4]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2,20)
net(X)
# 隐藏层256，2表示batchsize， 20表示特征数

tensor([[ 0.0533,  0.2307,  0.0299, -0.1066, -0.1040, -0.0982,  0.1624, -0.1134,
          0.0232, -0.2777],
        [ 0.0913,  0.1593, -0.0585, -0.1206,  0.0834,  0.0217,  0.0830, -0.0665,
         -0.1053, -0.2739]], grad_fn=<AddmmBackward>)

这里linear会自动初始w和b，这里为什么没有flatten，还没完全搞清楚flatten的作用
nn.Sequential 定义了一种特殊的Module

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__() # 调用父类，主要是初始化
        self.hidden = nn.Linear(20, 256) # linear 全连接层
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [13]:
net = MLP()
net(X) # 继承module，而module类的call会执行forword函数

tensor([[ 0.0103, -0.0004, -0.0759, -0.1301,  0.1702, -0.1276,  0.0418, -0.0915,
         -0.0290,  0.3180],
        [ 0.0216,  0.0712,  0.0018, -0.1916,  0.2107, -0.2853,  0.0186, -0.1828,
          0.0812,  0.2982]], grad_fn=<AddmmBackward>)

In [15]:
class MySequential(nn.Module):
        def __init__(self, *args): # *args收集list参数
            super().__init__() # 调用父类，主要是初始化
            for block in args:
                self._modules[block] = block # 这里self._modules 容器，放在里面的都是需要的层
        
        def forward(self, X):
            for block in self._modules.values():
                X = block(X)
            return X
        
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[-0.2111,  0.0379, -0.0567, -0.1172,  0.1223,  0.0130,  0.1463, -0.1801,
          0.0291,  0.1139],
        [-0.0873,  0.0444,  0.0732, -0.2042,  0.1200, -0.0756,  0.1535, -0.1658,
         -0.0282,  0.1223]], grad_fn=<AddmmBackward>)

In [17]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__() # 调用父类，主要是初始化
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() >1:
            X/=2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.1447, grad_fn=<SumBackward0>)

# 混合搭配各种组合块的方法

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0687, grad_fn=<SumBackward0>)